In [52]:
# Perform imports and load the dataset:
import numpy as np
import pandas as pd

df = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


# Check for missing value:

In [53]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

<div class="alert alert-info" style="margin: 20px">**There is no null value.**</div>

Check for the label value counts:

In [55]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

<font color=blue> 4825 out of 5572 messages, or 86.6%, are ham. This means that any text classification model we create has to perform better than 86.6% to beat random chance.</font>

Using train-test split to train the model and validate using test data:

In [57]:
from sklearn.model_selection import train_test_split

X= df['message']
y= df['label']

In [58]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.33, random_state=42)

## Scikit-learn's CountVectorizer
Text preprocessing, tokenizing and the ability to filter out stopwords are all included in [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), which builds a dictionary of features and transforms documents to feature vectors.

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3733, 7082)

<font color=blue>This shows that our training set is comprised of 3733 documents, and 7082 features.</font>

Now we will check the frequency of each word present in the docuument which is called as <font color=green>"Term Frequency"(tf)</font>.
But there will be more weight added to the common or stop words present in the documeent so to avoid weight issue we will use downscaling using <font color=green>"Inverse Document Frequency" (idf)</font>. 
It will downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This overall process comes under Tf-Idf transformation

In [60]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3733, 7082)

** The fit_transform performs two operations i.e. it fits an estimator to the data and then transforms our count-matrix to a tf-idf representation. **


<div class="alert alert-info" style="margin: 20px">We can combine the CountVectorizer and TfidTransformer steps into one using TfidVectorizer</div>

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(3733, 7082)

# Building Pipeline:


We will create a pipeline to vectorize the dataset into a full vocubalary and then train the model based on fully transformed data.

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])


# Testing classifiers:

In [64]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [65]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [66]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [68]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))


0.989668297988037


Using the text of the messages, our model performed exceedingly well; it correctly predicted spam **98.97%** of the time!<br>
Now let's apply what we've learned to a text classification project involving positive and negative movie reviews.
